In [1]:
import numpy as np
import random

In [2]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [3]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(13, 1), (13, 3), (9, 2), (9, 3), (11, 1), (14, 0), (13, 0), (10, 2)]
[(12, 2), (13, 2), (9, 1), (11, 0), (10, 1), (14, 3), (10, 0), (14, 2)]


In [4]:
class Player():
    def __init__(self, name):
        
        self.name = name
        self.cards = None
    
    ### -------------------------------------------------------------
    
    ### TO BE IMPLEMENTED - player's strategy 
    ### input: declared card, i.e., the card which is supposed
    ### to be the top card of the pile: If None - you can put any card you want because
    ### (a) it is the first turn (pile is empty) or (b) some cards were drawn in the previous turn)
    ### output: - player's true decision, player's declaration (if not equal - (s)he cheats)
    
    def putCard(self, declared_card):
        ### DO NOT REMOVE TRUE CARD cards.remove!!!
        ### return an object (not id): self.cards[id], not id
        ### for instance: return self.cards[0], self.cards[0] 
        ### IMPORTANT: If you want to draw cards instead of put, return "draw"
        ### for instance: return "draw" 
        return self.cards[0], self.cards[0] 
    
    ### TO BE IMPLEMENTED - Decide whether to check or not opponent's move (return True or False)
    def checkCard(self, opponent_declaration):
        pass
    
    ### Notification sent at the end of a round
    ### One may implement this method, capture data, and use it to get extra info
    ### -- checked = TRUE -> someone checked. If FALSE, the remaining inputs do not play any role
    ### -- iChecked = TRUE -> I decided to check my opponent (so it was my turn); 
    ###               FALSE -> my opponent checked and it was his turn
    ### -- iDrewCards = TRUE -> I drew cards (so I checked but was wrong or my opponent checked and was right); 
    ###                 FALSE -> otherwise
    ### -- revealedCard - some card (X, Y). Only if I checked.
    ### -- noTakenCards - number of taken cards
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " + 
                      str(iDrewCards) + "; revealed card = " + 
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))
    
    
    ### -------------------------------------------------------------
    
    ### Init player's hand
    def startGame(self, cards):
        self.cards = cards
    
    ### Add some cards to player's hand (if (s)he checked opponent's move, but (s)he was wrong)
    def takeCards(self, cards_to_take):
        self.cards = self.cards + cards_to_take

In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

In [6]:
class Game():
    def __init__(self, players, log = True):
        self.players = players
        self.deck = getDeck()
        self.player_cards = getShuffled(self.deck)
        self.game_deck = self.player_cards[0] + self.player_cards[1]
        
        self.cheats = [0, 0]
        self.moves = [0, 0]
        self.checks = [0, 0]
        self.draw_decisions = [0, 0]
        
        for i, cards in zip([0, 1], self.player_cards):
            self.players[i].startGame(cards.copy())
            if log:
                print("Player (" + str(i + 1) + "): " + self.players[i].name + " received:")
                print(self.players[i].cards)
        
        ### Which card is on top
        self.true_card = None
        ### Which card was declared by active player
        self.declared_card = None
        
        ### Init pile: [-1] = top card
        self.pile = []
        
        ### Which player moves
        self.player_move = np.random.randint(2)
        
    def takeTurn(self, log = True):
        
        self.player_move = 1 - self.player_move
        
        if log: 
            print("")
            print("")
            print("==== CURRENT STATE ================================")
            print("==== " + self.players[self.player_move].name + " MOVES ====")
            print("Player (0): " + self.players[0].name + " hand:")
            print(self.players[0].cards)
            print("Player (1): " + self.players[1].name + " hand:")
            print(self.players[1].cards)
            print("Pile: ")
            print(self.pile)
            print("Declared top card:")
            print(self.declared_card)
            print("")
            
        activePlayer = self.players[self.player_move]
        opponent = self.players[1 - self.player_move]
        self.moves[self.player_move] += 1
        
        self.previous_declaration = self.declared_card
        decision = activePlayer.putCard(self.declared_card)
        
        if decision == "draw":
            
            if log: print("[+] " + activePlayer.name + " decides to draw cards")
            
            self.draw_decisions[self.player_move] += 1
            
            toTake = self.pile[max([-3, -len(self.pile)]):]
            for c in toTake: self.pile.remove(c)
            activePlayer.takeCards(toTake)
            for c in toTake: self.player_cards[self.player_move].append(c)
            
            self.declared_card = None
            self.true_card = None
            
            activePlayer.getCheckFeedback(False, False, False, None, None, log)
            opponent.getCheckFeedback(False, False, False, None, None, log)

        else:
            self.true_card, self.declared_card = decision
            if self.true_card != self.declared_card: self.cheats[self.player_move] += 1
            
            if log: print("[+] " + activePlayer.name + " puts " + str(self.true_card) +
                          " and declares " + str(self.declared_card))
        
            if not self.debugMove(): return False, self.player_move
        
            activePlayer.cards.remove(self.true_card)
            self.player_cards[self.player_move].remove(self.true_card) 
            self.pile.append(self.true_card)
        
            if opponent.checkCard(self.declared_card):
                
                self.checks[1 - self.player_move] += 1
                
                if log: print("[!] " + opponent.name + ": " + "I want to check")
                toTake = self.pile[max([-3, -len(self.pile)]):]
                for c in toTake: self.pile.remove(c)

                if not self.true_card == self.declared_card:
                    if log: print("\tYou are right!")
                    activePlayer.takeCards(toTake)
                
                    activePlayer.getCheckFeedback(True, False, True, None, len(toTake), log)
                    opponent.getCheckFeedback(True, True, False, tuple(toTake[-1]), len(toTake), log)
                
                    for c in toTake: self.player_cards[self.player_move].append(c)
                else:
                    if log: print("\tYou are wrong!")
                    opponent.takeCards(toTake)  
                
                    activePlayer.getCheckFeedback(True, False, False, None, len(toTake), log)
                    opponent.getCheckFeedback(True, True, True, tuple(toTake[-1]), len(toTake), log)
               
                    for c in toTake: self.player_cards[1 - self.player_move].append(c)
            
                if log:
                    print("Cards taken: ")
                    print(toTake)

                self.declared_card = None
                self.true_card = None
            else:
                activePlayer.getCheckFeedback(False, False, False, None, None, log)
                opponent.getCheckFeedback(False, False, False, None, None, log)

            
        if not self.debugGeneral(): return False, self.player_move
        return True, self.player_move
            
    def isFinished(self, log = True):
        if len(self.players[self.player_move].cards) == 0:
            if log: print(self.players[self.player_move].name + " wins!")
            return True
        return False

    def debugMove(self):
        if (self.previous_declaration is not None) and (self.true_card[0] < self.previous_declaration[0]) and \
                    len(self.players[self.player_move].cards) == 1:
            print("[ERROR] Last played card should be valid (it is revealed, you cannot cheat)!")
            return False
        if np.array(self.true_card).size != 2: 
            print("[ERROR] You put too many cards!")
            return False
        if self.true_card not in self.player_cards[self.player_move]:
            print("[ERROR] You do not have this card!")
            return False
        if self.true_card not in self.deck:
            print("[ERROR] There is no such card!")
            return False
        if (self.previous_declaration is not None) and len(self.pile) == 0:
            print("[ERROR] Inconsistency")
            return False
        if (self.previous_declaration is not None) and (self.declared_card[0] < self.previous_declaration[0]):
            print(len(self.pile))
            print(self.previous_declaration)
            print(self.declared_card)
            print(self.pile[-1])
            print("[ERROR] Improper move!")
            return False
        return True
    
    def debugGeneral(self):
        A = set(self.players[0].cards)
        B = set(self.players[1].cards)
        C = set(self.player_cards[0])
        D = set(self.player_cards[1])
        P = set(self.pile)
        E = set(self.game_deck)
        
        if not A == C: 
            print("Error 001")
            return False
        if not B == D:
            print("Error 002")
            return False
        if not A | B | P == E:
            print("Error 003")
            print(A)
            print(B)
            print(P)
            print(E)
            return False
        return True

Analyze few moves...

In [7]:
player1 = RandomPlayer("Player A")
player2 = RandomPlayer("Player B")
game = Game([player1, player2])

for i in range(6):
    game.takeTurn()

Player (1): Player A received:
[(13, 1), (9, 2), (11, 2), (11, 0), (14, 3), (10, 3), (12, 3), (10, 2)]
Player (2): Player B received:
[(9, 0), (12, 0), (11, 3), (14, 1), (11, 1), (10, 1), (14, 2), (13, 0)]


==== CURRENT STATE ================================
==== Player A MOVES ====
Player (0): Player A hand:
[(13, 1), (9, 2), (11, 2), (11, 0), (14, 3), (10, 3), (12, 3), (10, 2)]
Player (1): Player B hand:
[(9, 0), (12, 0), (11, 3), (14, 1), (11, 1), (10, 1), (14, 2), (13, 0)]
Pile: 
[]
Declared top card:
None

[+] Player A puts (9, 2) and declares (9, 2)
[!] Player B: I want to check
	You are wrong!
Feedback = Player A : checked this turn = True; I checked = False; I drew cards = False; revealed card = None; number of taken cards = 1
Feedback = Player B : checked this turn = True; I checked = True; I drew cards = True; revealed card = (9, 2); number of taken cards = 1
Cards taken: 
[(9, 2)]


==== CURRENT STATE ================================
==== Player B MOVES ====
Player (0): Pla

In [8]:
### Some debug players
class DrawPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        return "draw"
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([False, False])
    
class SimplePlayer(Player):
    
    ### player's simple strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val: declaration = (min(min_val + 1, 14), declaration[1])
        return card, declaration
    
    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards: return True
        return np.random.choice([True, False], p=[0.3, 0.7])

In [14]:
### Perform a full game 100 times
stats_wins = [0, 0]
stats_moves = [0, 0]
stats_cheats = [0, 0]
stats_errors = [0, 0]
stats_cards = [0, 0]
stats_checks = [0, 0]
stats_draw_decisions = [0, 0]
stats_pile_size = 0

repeats = 100
errors = 0

for t in range(repeats):
    player1 = YourPlayer("Player A")
    player2 = SimplePlayer("Player B")
    game = Game([player1, player2], log = False)
    
    error = False
    while True:
        valid, player = game.takeTurn(log = False)
        if not valid:
            error = True
            stats_errors[player] += 1
            errors += 1
            break
        if game.isFinished(log = False):
            stats_wins[player] += 1
            break
            
    stats_pile_size += len(game.pile)
    if not error:
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

stats_pile_size /= (repeats - errors)          
for j in range(2):
    stats_moves[j] /= (repeats - errors)
    stats_cheats[j] /= (repeats - errors)
    stats_checks[j] /= (repeats - errors)
    stats_draw_decisions[j] /= (repeats - errors)
    stats_cards[j] /= (repeats - errors)

    
print("Wins:")
print(stats_wins)
print("Moves:")
print(stats_moves)
print("Cards:")
print(stats_cards)
print("Pile size:")
print(stats_pile_size)
print("Checks:")
print(stats_checks)
print("Draw decisions:")
print(stats_draw_decisions)
print("Cheats:")
print(stats_cheats)
print("Errors:")
print(stats_errors)
print("Total errors:")
print(errors)

Wins:
[85, 15]
Moves:
[9.96, 9.59]
Cards:
[1.86, 10.86]
Pile size:
3.28
Checks:
[3.2, 3.25]
Draw decisions:
[0.0, 0.0]
Cheats:
[1.7, 3.9]
Errors:
[0, 0]
Total errors:
0


In [9]:
### Implement player's strategy. You can compare it with random player 
### (or some strategy implemented by one of you colleagues)
### Time limit per decision 0.01s !!!

class YourPlayer(Player):
    
    def __init__(self, name):
        #(100,100) oznacza niewiadomą kartę
        self.opponentCards=[(100,100),(100,100),(100,100),(100,100),(100,100),(100,100),(100,100),(100,100)] 
        self.noneCard=True
        self.stack=[]
        self.Checked=False
        self.decision=2
        self.allCards=[]
        self.first=True
        self.cheating=False
        self.chosenCard=[]
    
    def putCard(self, declared_card):
        #jeżeli jest to pierwsza runda, to wszystkie karty, które mamy na ręcę zapamiętujemy, że są w grze
        if self.first:
            self.first=False
            self.allCards=self.cards.copy()      
        self.noneCard=False    
        
        #jeżeli możemy położyć dowolną kartę, nie jest to pierwsza tura i nie sprawdzaliśmy w tej turze, to znaczy, że przeciwnik wykonał akcję 'draw'
        #zatem z zapamiętanej listy kart ze stosu usuwamy 3 karty i dokładamy je do kart przeciwnika
        if not declared_card and len(self.stack)>0 and not self.Checked:
            n=min(len(self.stack),3)
            for i in range(n):    
                self.opponentCards.append(self.stack[len(self.stack)-1])
                self.stack.pop(len(self.stack)-1)
        self.Checked=False
        
        #wybieramy najniższą kartę z ręki
        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            #jeżeli nie możemy położyć tej karty, to pewnie będziemy kłamać...
            if card[0] < min_val: 
                i=0
                self.cheating=False
                p1=0.7 #prawdopodobieństwo skłamania
                #sprawdzamy, czy mamy na ręce kartę o tym samym nr, co leży na stosie
                #jeśli tak, to losujemy, czy skłamać kładąc naszą najniższą kartę jako tą kartę
                if (min_val,0) in self.cards:
                    self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                    if self.cheating: self.chosenCard=(min_val,0)
                elif (min_val,1) in self.cards:
                    self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                    if self.cheating: self.chosenCard=(min_val,1)
                elif (min_val,2) in self.cards:
                    self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                    if self.cheating: self.chosenCard=(min_val,2)
                elif (min_val,3) in self.cards: 
                    self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                    if self.cheating: self.chosenCard=(min_val,3)
                
                if not self.cheating:
                    #jeżeli nie skłamaliśmy, możemy skłamać kładąc kartę o 1 wyższą od karty na stosie, którą mamy na ręce
                    j=declared_card[0]+1
                    for f,c in self.cards:
                        if f==j:
                            if not self.cheating:
                                self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                                self.chosenCard=(f,c)
                                
                #jeżeli nie skłamaliśmy:
                if not self.cheating:
                    self.chosenCard=self.opponentCards[0]
                    inAllCards=True
                    #losujemy kartę do 2 nr wyższą od obecnej na stosie, sprawdzamy, czy nie ma jej przeciwnik lub nie jest na stosie
                    #jeżeli znamy już wszystkie karty w grze, to sprawdzamy też, czy znajduje się wśród tych kart
                    while (self.chosenCard in self.opponentCards or self.chosenCard in self.stack or not inAllCards) and i<100: 
                        pom=random.randrange(4)
                        higher=random.randrange(3)
                        self.chosenCard=(min(min_val+higher,14),pom)
                        if len(self.allCards)==24:
                            inAllCards=False
                            if self.chosenCard in self.allCards:
                                inAllCards=True
                        i+=1
                    #losujemy, czy mamy w tym momencie skłamać
                    if i<100:
                        self.cheating=np.random.choice([True, False], p=[0.8,0.2])
                
                #deklarujemy wybraną kartę, że kładziemy na stół
                if self.cheating or i<100:
                    declaration = self.chosenCard

                #jeżeli nie kłamiemy, kładziemy najniższą możliwą kartę
                if not self.cheating:
                    mini=15
                    for f,c in self.cards:
                        if f>=declared_card[0] and f<mini:
                            declaration=(f,c)
                            card=(f,c)
                            mini=f
                
        self.stack.append(card)
        return card, declaration
    
    def checkCard(self, opponent_declaration):
        minCard = min(self.opponentCards, key=lambda x: x[0])
        
        #jeżeli przeciwnik kładzie kartę, którą podejrzewamy, że ma na ręce, nie będziemy sprawdzać
        if opponent_declaration in self.opponentCards:
            self.stack.append(opponent_declaration)
            self.opponentCards.remove(opponent_declaration)
            self.decision=0
        #jeżeli przeciwnik ma na ręce karty, których nie znamy
        elif (100,100) in self.opponentCards:
            self.stack.append((100,100))
            self.opponentCards.remove((100,100))
            self.decision=2
        #będziemy sprawdzać, bo przeciwnik kładzie kartę, której zapewne nie ma
        else:
            self.stack.append((100,100))
            self.opponentCards.remove(minCard)
            self.decision=1
        
        if opponent_declaration in self.cards: return True
        #sprawdzamy zawsze, kiedy zostanie nam na ręce 1 karta o nr niższym, niż ten na stosie
        #lepiej jest sprawdzić, niż brać 3 karty
        if len(self.cards) == 1 and self.cards[0][0] < opponent_declaration[0]: return True
        #jeżeli mamy 1 kartę, którą możemy położyć, to nie ma sensu sprawdzać
        if len(self.cards) == 1 and self.cards[0][0] >= opponent_declaration[0]: return False
        #sprawdzamy przeciwnika, jeśli nie kładł dowolnej karty, znamy przynajmniej 15 kart w grze i nie kładzie żadnej z nich
        if not self.noneCard and len(self.allCards)>=15 and opponent_declaration not in self.allCards: return True
        #jeśli przeciwnik kładł dowolną kartę, to sprawdzamy tylko jeśli karta ta ma nr 13 lub 14 i mamy co najmniej 3 karty na ręce
        if self.noneCard and opponent_declaration[0]<13: return False
        if self.noneCard and opponent_declaration[0]>=13: 
            if len(self.cards)<3: return False
            else: return True
        if self.decision==0: return False
        if self.decision==1: return True  
        #jeżeli znamy wszystkie karty przeciwnika i karta, którą kładzie ma nr>=najmniejszego nr na ręce, to nie sprawdzamy
        if (100, 100) not in self.opponentCards and minCard[0]>=opponent_declaration[0]: return False
        #jeśli podejrzewamy, że karta, którą położył przeciwnik już szła, to sprawdzamy
        if opponent_declaration in self.stack: return True
        
        #jeżeli mamy 2 karty na ręce, to niezbyt opłaca się sprawdzać, więc prawdopodobieństwo jest małe
        if len(self.cards)<3:
            return np.random.choice([True, False], p=[0.1, 0.9])
        #jeśli przeciwnik ma mało kart, to go sprawdzamy
        if len(self.opponentCards)<3:
            return np.random.choice([True, False], p=[0.8, 0.2])
        #jeżeli przeciwnik kładzie kartę o nr 9 lub 10, to raczej mówi prawdę
        if opponent_declaration[0]<11:
            return np.random.choice([True, False], p=[0.05, 0.95])
        #jeżeli wiemy, że przeciwnik ma na ręce kartę niższą od obecnej na stosie, to prawdopodobieństwo sprawdzenia rośnie
        if opponent_declaration[0]>=minCard[0]+4:
            return np.random.choice([True, False], p=[0.7, 0.3])
        if opponent_declaration[0]>=minCard[0]+3:
            return np.random.choice([True, False], p=[0.6, 0.4])
        if opponent_declaration[0]>=minCard[0]+2:
            return np.random.choice([True, False], p=[0.5, 0.5])
        #prawdopodobieństwo sprawdzenia jest tym większe, im więcej znamy kart w grze
        if opponent_declaration not in self.allCards:
            return np.random.choice([True, False], p=[len(self.allCards)/24, 1-len(self.allCards)/24])
        else:
            return np.random.choice([True, False], p=[0.2,0.8])
    
    #zapisujemy informacje o tym kto i jakie bierze karty, jakie karty sa usuwane ze stosu
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        if checked:
            if not iDrewCards:
                if iChecked:
                    self.opponentCards.append(revealedCard)
                    self.stack.pop(len(self.stack)-1)
                    noTakenCards-=1
                    if revealedCard not in self.allCards:
                        self.allCards.append(revealedCard)
                for n in range(noTakenCards):
                    self.opponentCards.append(self.stack[len(self.stack)-1])
                    self.stack.pop(len(self.stack)-1)
            else:
                del self.stack[len(self.stack)-3 : len(self.stack)-1] 
            
            if iChecked:
                self.Checked=True
            else:
                self.noneCard=True
            
    def takeCards(self, cards_to_take):
        self.cards = self.cards + cards_to_take
        for c in cards_to_take:
            if c not in self.allCards:
                self.allCards.append(c)